In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/raw/creditcard.csv")

In [3]:
df.shape

(284807, 31)

In [4]:
df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [5]:
df.duplicated().sum()

np.int64(1081)

In [6]:
dup_rows = df.duplicated().sum()
dup_rows

np.int64(1081)

In [7]:
df.shape

(284807, 31)

In [8]:
df.dtypes

Time      float64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
Class       int64
dtype: object

# Duplicate Handling
Identified 1,081 exact duplicate transaction rows in the raw dataset, including both fraudulent and legitimate transactions. Removed duplicates to avoid inflating transaction volume and fraud metrics used in downstream analysis.

In [9]:
df[df.duplicated(keep=False)]["Class"].value_counts()

Class
0    1822
1      32
Name: count, dtype: int64

In [10]:
rows_before =  df.shape[0]

df = df.drop_duplicates()

rows_after = df.shape[0]

rows_before, rows_after, rows_before - rows_after

(284807, 283726, 1081)

In [11]:
df.duplicated().sum()

np.int64(0)

In [12]:
df["Class"].value_counts()

Class
0    283253
1       473
Name: count, dtype: int64

In [13]:
df = df.rename(columns={
    "Class": "is_fraud",
    "Time": "time_seconds"})

In [14]:
df.columns

Index(['time_seconds', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9',
       'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19',
       'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'is_fraud'],
      dtype='object')

# Creating TIME-based derived columns

In [15]:
df["transaction_hour"] = (df["time_seconds"]//3600).astype(int)

In [16]:
df["transaction_day"] = (df["time_seconds"]//86400).astype(int)

In [17]:
def time_bucket(hour):
    if hour < 6:
        return "Early"
    elif hour < 12:
        return "Morning"
    elif hour < 18:
        return "Afternoon"
    else:
        return "Evening"

df["time_bucket"] = df["transaction_hour"].apply(time_bucket)

In [21]:
df[["transaction_hour", "transaction_day", "time_bucket"]].describe()

,transaction_hour,transaction_day
count,283726.000000,283726.000000
mean,25.844917,0.491636
std,13.182881,0.499931
min,0.000000,0.000000
25%,15.000000,0.000000
50%,23.000000,0.000000
75%,38.000000,1.000000
max,47.000000,1.000000


In [20]:
df["time_bucket"].value_counts()

time_bucket
Evening      189802
Afternoon     46731
Morning       34897
Early         12296
Name: count, dtype: int64

# Create AMOUNT-based derived columns

In [22]:
low_threshold = df["Amount"].quantile(0.25)
high_threshold = df["Amount"].quantile(0.75)

low_threshold, high_threshold

(np.float64(5.6), np.float64(77.51))

In [23]:
def amount_bucket(amount):
    if amount < low_threshold:
        return "Low"
    elif amount <= high_threshold:
        return "Medium"
    else:
        return "High"

df["amount_bucket"] = df["Amount"].apply(amount_bucket)

In [24]:
df["is_high_amount"] = df["Amount"] >  high_threshold

In [26]:
df["amount_bucket"].value_counts()

amount_bucket
Medium    141895
High       70931
Low        70900
Name: count, dtype: int64

In [27]:
df["is_high_amount"].value_counts()

is_high_amount
False    212795
True      70931
Name: count, dtype: int64

# Create fraud-friendly labels

In [28]:
df["fraud_label"] = df["is_fraud"].apply(
    lambda x: "Fraud" if x == 1 else "Legitimate"
)

In [29]:
df["fraud_label"].value_counts()

fraud_label
Legitimate    283253
Fraud            473
Name: count, dtype: int64

In [30]:
df.shape

(283726, 37)

In [31]:
df.isnull().sum().sort_values(ascending = False).head()

time_seconds    0
V19             0
V21             0
V22             0
V23             0
dtype: int64

In [32]:
df.head()

,time_seconds,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V27,V28,Amount,is_fraud,transaction_hour,transaction_day,time_bucket,amount_bucket,is_high_amount,fraud_label
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,0.133558,-0.021053,149.62,0,0,0,Early,High,True,Legitimate
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.008983,0.014724,2.69,0,0,0,Early,Low,False,Legitimate
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,-0.055353,-0.059752,378.66,0,0,0,Early,High,True,Legitimate
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,0.062723,0.061458,123.50,0,0,0,Early,High,True,Legitimate
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,0.219422,0.215153,69.99,0,0,0,Early,Medium,False,Legitimate


In [33]:
df.to_csv("../data/processed/fraud_analysis_clean.csv")